In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
from data_agent import *
import numpy as np
from data_util import *
import tensorflow as tf
from sklearn_model_hyper import *
import pandas as pd
from stock_agent import *
from backtest import *
from joblib import dump, load
from datetime import datetime, timedelta
from tec_an import *
from bitstamp import *


path = "./data/"


In [2]:
def load_online(minutes, window, val_end = 1626388860):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)


    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = ["btcusd"],
                 train_limit = 40,
                 val_limit = 400,
                 val_keys = ["btcusd"],
                 #val_start = val_start,
                 val_end = val_end,
                 train_start_list = [1626386301]
    )

    online.load_cache()
    return online

In [3]:
online = load_online(3, 20, 1626459248)

valX, valY = online.load_val_data("btcusd")



https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 180, 'limit': 40, 'start': 1626386301}
Downloaded from 2021-07-15 22:57:00 to 2021-07-16 00:54:00 31644.19
Resample 3Min - TecAn ( windows 20, windows_limit 100 )
Processing 39 of btcusd


  0%|          | 0/39 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 180, 'limit': 400, 'start': 1623435037, 'end': 1626459248}
Downloaded from 2021-07-15 23:15:00 to 2021-07-16 19:12:00 31977.18
Resample 3Min - TecAn ( windows 20, windows_limit 100 )
Processing 399 of btcusd


  0%|          | 0/399 [00:00<?, ?it/s]

In [4]:
model = load(f"model/min-{3}-window-{20}")
print(model)

Pipeline(steps=[('s', RFECV(estimator=RandomForestClassifier(random_state=10))),
                ('m',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])


In [5]:
back = BackTest(value = 100,
                    verbose = True, 
                    sell_on_profit = True)

request_sell = lambda price: back.request_sell(price)
request_buy = lambda price: back.request_buy(price)
on_state = lambda timestamp, price: back.on_state(timestamp, price)

stock = StockAgent(
    model = model,
    request_sell = request_sell,
    request_buy = request_buy,
    on_state = on_state
)


on_new_data = lambda x: print(x)
on_new_data = lambda x: stock.on_x(x)

on_state = lambda timestamp, price, buy, sell: print("{} {} {} {}".format(timestamp, price, buy, sell))
on_state = lambda timestamp, price, buy, sell: stock.on_new_state(timestamp, price, buy, sell)

agent = DataAgent(
    taProc = TacProcess(), 
    tec = TecAn(windows = 20, windows_limit = 100),
    resample = '3Min',
    on_state = on_state,
    on_new_data = on_new_data
)


TIMESTAMP_KEY = "timestamp"

Resample 3Min - TecAn ( windows 20, windows_limit 100 )


In [6]:
for yy in valY:
    #print(yy)
    agent.taProc.add_tacs_realtime([], yy, 0.0, agent.tec)

In [7]:
#start = int(datetime.timestamp((datetime.now() + timedelta( hours=-5 ))))



In [19]:
raw_data_live = []


class RawStateDownloader(LiveBitstamp):
    
    def __init__(self, on_raw_data = lambda raw: print(raw)):
        self.trade = {}
        self.on_new_data_count = 0
        self.on_raw_data = on_raw_data
                
    def process(self, raw):
        #raw_data_live.append(raw)
        self.on_raw_data(raw)
        timestamp = raw[TIMESTAMP_KEY]
        time = pd.to_datetime(timestamp, unit='s')
        print(f'{time}: {agent.on_new_data_count} {len(raw_data_live)}', end='\r')
        

In [20]:

on_raw_data = lambda raw: agent.on_new_raw_data(raw)


live = RawStateDownloader(on_raw_data = on_raw_data)


In [ ]:
back.reset()

bt = Bitstamp(live, currency = "btcusd")

bt.connect()

Bitstamp Websocket Opened.
Reading form live_trades_btcusd and order_book_btcusd
2021-07-16 19:15:16 Buy (31992.81): ask: 31992.81
2021-07-16 19:18:02 Sell (32025.62) profit: 0.1026 total: 100.10255429266763
2021-07-16 19:27:01 Buy (32021.93): ask: 32021.93
#### LOSSSS: -0.3013 26 0
2021-07-16 19:36:11 Sell (31925.45) profit: -0.3013 total: 99.80126077288291
2021-07-16 19:45:11 Buy (31877.12): ask: 31877.12
2021-07-16 19:48:00 Sell (31907.21) profit: 0.0944 total: 99.89565449477497
2021-07-16 19:51:07 Buy (31892.8): ask: 31892.8
2021-07-16 19:57:04 Sell (31913.64) profit: 0.0653 total: 99.96099839684692
2021-07-16 20:00:02 Buy (31900.0): ask: 31900.0
2021-07-16 20:03:00 Sell (31969.02) profit: 0.2164 total: 100.17736203321056
2021-07-16 20:06:15 Buy (31940.68): ask: 31940.68
#### LOSSSS: -0.0322 42 0
2021-07-16 20:24:00 Sell (31930.41) profit: -0.0322 total: 100.14520867893006
2021-07-16 20:27:01 Buy (31932.91): ask: 31932.91
#### LOSSSS: -0.0424 45 0
2021-07-16 20:33:07 Sell (31919.36

2021-07-17 10:18:10 Buy (31349.78): ask: 31349.78
#### LOSSSS: -0.0049 321 0
2021-07-17 10:24:03 Sell (31348.24) profit: -0.0049 total: 97.70460500507352
2021-07-17 11:06:06 Buy (31332.72): ask: 31332.72
2021-07-17 11:12:09 Sell (31395.7) profit: 0.201 total: 97.90560893961863
2021-07-17 11:24:05 Buy (31329.65): ask: 31329.65
2021-07-17 11:30:13 Sell (31330.15) profit: 0.0016 total: 97.90720487190642
2021-07-17 11:33:03 Buy (31352.3): ask: 31352.3
2021-07-17 11:39:07 Sell (31375.21) profit: 0.0731 total: 97.98027766082461
2021-07-17 11:42:12 Buy (31396.64): ask: 31396.64
#### LOSSSS: -0.0205 349 0
2021-07-17 11:48:21 Sell (31390.21) profit: -0.0205 total: 97.95979776233865


In [ ]:
82

In [ ]:
back.report()

In [13]:
print(raw_data_live)

[]


In [ ]:
date.timestamp()